In [33]:
import os
import glob
import uproot
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 150

import operator
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

from ipywidgets import interact, interactive, fixed, interact_manual
from ordered_set import OrderedSet
from plotting_utils import get_results#, plot_results

%matplotlib inline

In [29]:
def plot_results(results, name, log):

    vevds = OrderedSet([r.vevd for r in sorted(results, key=operator.attrgetter('vevd'), reverse=True)])
    mdpps = OrderedSet([r.mdpp for r in sorted(results, key=operator.attrgetter('mdpp'))])

    hmap = []
    
    print(vevds)

    for v in vevds:
        results_msorted = sorted(results, key=operator.attrgetter('mdpp'))
        hmap.append([r.quantity for r in results_msorted if r.vevd == v])
        
    hmap = np.array(hmap, dtype=float)

    fig, ax = plt.subplots()
    if log:
        sns.heatmap(hmap, annot=True, cmap="Oranges", xticklabels=mdpps, yticklabels=vevds, ax=ax,
                    fmt=".2", norm=LogNorm(), annot_kws={"size": 15 / np.sqrt(len(hmap))},
                    cbar_kws={'label': name});
    if not log:
        sns.heatmap(hmap, annot=True, cmap="Oranges", xticklabels=mdpps, yticklabels=vevds, ax=ax,
                    annot_kws={"size": 15 / np.sqrt(len(hmap))},
                    cbar_kws={'label': name});

    ax.set_xlabel(r"$M_{\Delta^{\pm\pm}}$ [GeV]");
    ax.set_ylabel(r"vevD [GeV]");

In [30]:
def select(sample_folder, variable, log):
    results = get_results(sample_folder, variable)
    plot_results(results, variable, log)

runs = glob.glob("../multirun/*")
choices = ["total_xs", "total_xs_error", "sum_of_weights", "sum_of_weights_err", "fid_sum_of_weights", "fid_sum_of_weights_err", "fid_xs", "fid_xs_err", "nsurvive_events", "ninit_events",  "presel_eff",   "cutflow"]

interact(select, sample_folder=runs, variable=choices, log=[True, False]);

interactive(children=(Dropdown(description='sample_folder', options=('../multirun/2023-03-13_typeII_single_pro…

In [39]:
def select_folder(folder):
    global selected_folder
    selected_folder = folder


def get_vevs_and_masses(sample_folder):
    folders = [p for p in glob.glob(f"{sample_folder}/*") if os.path.isdir(p)]
    vevds = []
    mdpps = []
    for fpath in folders:
        split_path = os.path.basename(fpath).split("_")
        vevds.append(float(split_path[-1]))
        mdpps.append(float(split_path[-2]))
    
    vevds.sort()
    mdpps.sort()
    vevds = OrderedSet(vevds)
    mdpps = OrderedSet(mdpps)
    
    return vevds, mdpps


def get_root_file_path(mass, vevd):
    folders = [p for p in glob.glob(f"{selected_folder}/*") if os.path.isdir(p)]
    for fpath in folders:
        split_path = os.path.basename(fpath).split("_")
        if float(split_path[-1]) == vevd and float(split_path[-2]) == mass:
            return f"{fpath}/output.root"

def plot_sample(mass, vevd):
    root_filepath = get_root_file_path(mass, vevd)

    data = uproot.open(f"{root_filepath}:tree")
    for observable in data.keys():
        array = data[observable].array(library='ak')
        array = np.ravel(array)
        fig, ax = plt.subplots()
        ax.hist(array, 50, label=f'mass = {mass}  vevd={vevd}', histtype='step')
        ax.set_xlabel(observable)
        ax.legend()
        plt.show()


interact(select_folder, folder=glob.glob("../multirun/*/"));

vevds, mdpps = get_vevs_and_masses(selected_folder)

interact(plot_sample, mass=mdpps, vevd=vevds);

interactive(children=(Dropdown(description='folder', options=('../multirun/2023-03-13_typeII_single_prod_lljj_…

interactive(children=(Dropdown(description='mass', options=(100.0, 300.0, 500.0, 700.0, 900.0, 1100.0, 1300.0,…